<a href="https://colab.research.google.com/github/veiro/Extentend_Data_imputation_with_CTGAN/blob/main/GANS_tarea_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nov 1, 2019

Conditional Imputation GAN on synthetic ranking data. Please do not distribute.



### Reading Data

In [ ]:
!pip install sdv
# la primera vez da error, reiniciar ambiente y ejecutar todo de nuevo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sdv.tabular import CTGAN
from ctgan.data_transformer import DataTransformer
from ctgan.data_sampler import DataSampler

In [ ]:
import os
import sys

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import pandas
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
%%time
data_filepath = 'https://raw.githubusercontent.com/gdeng96/cond-imp-gan-ranking/main/synthetic_ranking_data.csv'
data_full = pandas.read_csv(data_filepath)

print("Cantidad de datos reales:")
print(data_full.shape)

data =  data_full.sample(n=100000, random_state=1)
#data =  data_full[0:100000] #.sample(n=100000, random_state=1)

print("Cantidad de datos sampleados:")
print(data.shape)

Cantidad de datos reales:
(640000, 9)
Cantidad de datos sampleados:
(100000, 9)
CPU times: user 939 ms, sys: 185 ms, total: 1.12 s
Wall time: 6.84 s


### Preparing Data

In [ ]:
RANKING_COLS = ["V5", "V6", "V7", "V8", "V9"]
dataToTransform = data.loc[:, RANKING_COLS]

In [ ]:
%%time
dt = DataTransformer()
discrete_columns = ['V8']

dt.fit(raw_data=dataToTransform, discrete_columns=discrete_columns)
dataTransformed = dt.transform(raw_data=dataToTransform)
#sampler = DataSampler(data=dataTransformed, output_info=dt.output_info_list, log_frequency=True)
#condvec = sampler.sample_condvec(batch=10)

CPU times: user 34.6 s, sys: 29.2 s, total: 1min 3s
Wall time: 1min


In [ ]:
df2 = pd.DataFrame(dataTransformed)

df2.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)

dataTotal= pd.concat([data, df2], axis=1)



In [ ]:
dataTotal

,Category,QID,ProductID,QProductID,V5,V6,V7,V8,V9,0,...,57,58,59,60,61,62,63,64,65,66
0,Books,Books634,30,Books634-30,0.575460,2.181218,0.361658,2,2.017008,-0.143654,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Beauty,Beauty922,21,Beauty922-21,2.826844,10.096474,0.082438,3,19.432840,-0.426745,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Clothes,Clothes1598,30,Clothes1598-30,5.150209,0.739852,0.028061,14,30.689661,0.382142,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Electronics,Electronics1691,52,Electronics1691-52,3.108515,5.493077,0.773000,12,12.173721,-0.264248,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Beauty,Beauty1823,50,Beauty1823-50,3.272858,1.138467,1.360502,4,11.082728,-0.445619,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Books,Books1315,40,Books1315-40,-1.760588,4.236946,4.033930,2,0.654007,-0.199840,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,Clothes,Clothes1391,34,Clothes1391-34,2.981975,1.346800,0.429989,5,31.436584,-0.337250,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,Electronics,Electronics1152,55,Electronics1152-55,4.086355,4.320428,1.167587,13,15.418107,0.299873,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
99998,Clothes,Clothes1665,58,Clothes1665-58,2.674404,1.387580,1.151788,9,0.226623,-0.108613,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataTotal['Category'].unique()

array(['Books', 'Beauty', 'Clothes', 'Electronics', 'Furniture'],
      dtype=object)

In [ ]:
#trainlabels = pandas.get_dummies(dataTotal.Category, prefix='Category')

In [ ]:
#df2 = pd.DataFrame(trainlabels)
#dataTotal= pd.concat([dataTotal, df2], axis=1)

In [ ]:
trainData, testData = train_test_split(dataTotal, test_size=0.10, random_state=41)

In [ ]:
DATASET_COLS = ["V5", "V6", "V7", "V8", "V9", "Category", "QID", "ProductID", "QProductID", 'Category_Beauty', 'Category_Books', 'Category_Clothes', 'Category_Electronics', 'Category_Furniture']
cols = [x for x in dataTotal.columns if (x not in DATASET_COLS)]
trainX = trainData.loc[:, cols]
testX = testData.loc[:,cols]

trainX = trainX.values
testX = testX.values

In [ ]:
print(trainX.shape)
print(testX.shape)

Train_No = trainX.shape[0] 
Test_No = testX.shape[0]

(90000, 67)
(10000, 67)


In [ ]:
Train_No

90000

In [ ]:
Test_No

10000

In [ ]:
# Number of ranking features that needs imputing
Dim = trainX.shape[1] 
print("Dim :" + str(Dim))

Dim :67


In [ ]:
nClass = trainData['Category'].unique().size 

In [ ]:
##Create one-hot conditional matrix - numpy array
trainlabelsX = pandas.get_dummies(trainData.Category, prefix='Category')
testlabelsX = pandas.get_dummies(testData.Category, prefix='Category')
trainlabelsX = trainlabelsX.values
testlabelsX = testlabelsX.values

## Train GAN

In [ ]:
# Set Hyperparameters
# Mini-batch
mb_size = 256
# Iterations
n_epoch = 500
# Missing rate
p_miss = 0.05
# Fully-connected layer sizes (change depending on Dim)
col1 = 64 
col2 = 64 

In [ ]:
# Other functions
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
  
trainM = sample_M(Train_No, Dim, p_miss)
testM = sample_M(Test_No, Dim, p_miss)

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)

def sample_Z(m, n):
    return np.random.uniform(0., 0.1, size = [m, n])        

def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    idx
    return idx


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#Tensors
X = tf.placeholder(tf.float32, shape = [None, Dim])
M = tf.placeholder(tf.float32, shape = [None, Dim])
H = tf.placeholder(tf.float32, shape = [None, Dim])
Z = tf.placeholder(tf.float32, shape = [None, Dim])
C = tf.placeholder(tf.float32, shape = [None, nClass])

D_W1 = tf.Variable(xavier_init([Dim*2 + nClass, col1]))    
D_b1 = tf.Variable(tf.zeros(shape = [col1]))
D_W2 = tf.Variable(xavier_init([col1, col2]))
D_b2 = tf.Variable(tf.zeros(shape = [col2]))
D_W3 = tf.Variable(xavier_init([col2, Dim]))
D_b3 = tf.Variable(tf.zeros(shape = [Dim]))   

G_W1 = tf.Variable(xavier_init([Dim*2 + nClass, col1]))     
G_b1 = tf.Variable(tf.zeros(shape = [col1]))
G_W2 = tf.Variable(xavier_init([col1, col2]))
G_b2 = tf.Variable(tf.zeros(shape = [col2]))
G_W3 = tf.Variable(xavier_init([col2, Dim]))
G_b3 = tf.Variable(tf.zeros(shape = [Dim]))

In [ ]:
%%time 
#Setting up conditional generator
def generator_conditional(x,z,m,c):
    inp = m * x + (1-m) * z  
    inputs = tf.concat(axis = 1, values = [inp,c,m])  
    print("G input: " + str(inputs.shape))
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    G_est = tf.matmul(G_h2, G_W3) + G_b3
    return G_est

    
#Setting up conditional discriminator
def discriminator_conditional(x, m, g, h, c):
    inp = m * x + (1-m) * g  
    inputs = tf.concat(axis = 1, values = [inp,c,h])  
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)  
    return D_prob

#Generate fake copies
print("X: " + str(X.shape))
print("Z: " + str(Z.shape))
print("M: " + str(M.shape))
print("C: " + str(C.shape))

G_sample = generator_conditional(X,Z,M,C)
D_prob = discriminator_conditional(X, M, G_sample, H,C)

D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) * 2
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8)) / tf.reduce_mean(1-M)
MSE_train_loss = tf.reduce_mean((M * X - M * G_sample)**2) / tf.reduce_mean(M)
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

D_loss = D_loss1 + MSE_test_loss
G_loss = G_loss1 + 10 * MSE_train_loss 


#Imputed Copy
Imputed_copy = M * X + (1-M) * G_sample

#Adam Optimizer
D_params = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
G_params = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_params)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_params)

X: (?, 67)
Z: (?, 67)
M: (?, 67)
C: (?, 5)
G input: (?, 139)
CPU times: user 620 ms, sys: 18.6 ms, total: 639 ms
Wall time: 637 ms


In [ ]:
%%time
sess = tf.Session()
sess.run(tf.global_variables_initializer())


i = 1
DiscriminatorLoss = []
GeneratorLoss = []
IterationsRecord = []

tqdm(range(n_epoch))

for it in tqdm(range(n_epoch)):
    
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-0.5)
    H_mb = M_mb * H_mb1
    C_mb = trainlabelsX[mb_idx, :]
    
    #print("M_mb: " + str(M_mb.shape))
    #print("X_mb: " + str(X_mb.shape))
    #print("Z_mb: " + str(Z_mb.shape))
    #print("H_mb: " + str(H_mb.shape))
    #print("C_mb: " + str(C_mb.shape))

    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss = sess.run([D_solver, D_loss1], feed_dict = {X: X_mb, M: M_mb, Z: New_X_mb, H: H_mb, C: C_mb})
    _, G_loss, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, Z: New_X_mb, H: H_mb, C: C_mb})
         
        
    #%% Intermediate Losses
    if it % 50 == 0:
        IterationsRecord.append(it)
        DiscriminatorLoss.append(D_loss)
        GeneratorLoss.append(G_loss)
        print('Iter: {}'.format(it))
        print('Dloss: {:.4}'.format(D_loss))
        print('Gloss: {:.4}'.format(G_loss))
        print()



  0%|          | 0/500 [00:00<?, ?it/s]

M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)


  2%|▏         | 10/500 [00:03<01:47,  4.55it/s]

Iter: 0
Dloss: 1.71
Gloss: 0.8228

M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X

  6%|▋         | 32/500 [00:03<00:24, 19.10it/s]

...

M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb

 99%|█████████▊| 493/500 [00:08<00:00, 96.64it/s]

M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb

100%|██████████| 500/500 [00:08<00:00, 59.48it/s]

M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
M_mb: (256, 67)
X_mb: (256, 67)
Z_mb: (256, 67)
H_mb: (256, 67)
C_mb: (256, 5)
CPU times: user 6.05 s, sys: 732 ms, total: 6.79 s
Wall time: 8.5 s


### Record some summary statistics


In [ ]:
#Getting imputations for test ranking dataset + RMSE.
N_imputations = 10
RMSEoutput = []

for j in range(N_imputations):
    Z_mb = sample_Z(Test_No, Dim) 
    M_mb = testM
    X_mb = testX
        
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb       
    New_C_mb = testlabelsX
    
    MSE_final, Sample, Imputed_copy_v2, M_v2 = sess.run([MSE_test_loss, G_sample, Imputed_copy, M], feed_dict = {X: testX, M: testM, Z: New_X_mb, C: New_C_mb})
   
    print("MSE_final: " + str(MSE_final))
    RMSEoutput.append(np.sqrt(MSE_final))
    print("Done with imputation", j)


MSE_final: 0.07101627
Done with imputation 0
MSE_final: 0.07095797
Done with imputation 1
MSE_final: 0.07094114
Done with imputation 2
MSE_final: 0.0709452
Done with imputation 3
MSE_final: 0.070948
Done with imputation 4
MSE_final: 0.07094033
Done with imputation 5
MSE_final: 0.07098082
Done with imputation 6
MSE_final: 0.071020186
Done with imputation 7
MSE_final: 0.070980154
Done with imputation 8
MSE_final: 0.07101836
Done with imputation 9


In [ ]:
RMSEDF = pandas.DataFrame({'RMSE': RMSEoutput})
RMSEDF.to_csv('rmse.csv', index=False, header=True)

In [ ]:
RMSEDF.mean(axis=0)

RMSE    0.266411
dtype: float32

Code adapted based on Yoon et al. (2018)



